In [1]:
#!g1.1
use_koila = True
use_resnet = False
fc_hidden = 10000

In [2]:
#!g1.1
%matplotlib inline

In [3]:
#!g1.1
!python3 --version

Python 3.8.12


In [4]:
#!g1.1
%pip install -U pip -q

In [5]:
#!g1.1
%pip install wandb -q

In [6]:
#!g1.1
%pip install pynvml -q

In [7]:
#!g1.1
%pip install koila -q

In [8]:
#!g1.1
import numpy as np
import torch
import wandb
import torch.nn as nn
import koila
from koila import LazyTensor, lazy

In [9]:
#!g1.1
import torchvision
import torchvision.transforms as transforms

In [10]:
#!g1.1
# max_memory_gib = torch.cuda.get_device_properties('cuda').total_memory / 2 ** 30
# torch.cuda.set_per_process_memory_fraction(min(1.0, 11 / max_memory_gib))
# print(f"Setting memory limit to {min(1.0, 11 / max_memory_gib) * 100:.2f}%")

In [11]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [12]:
#!g1.1
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: i_pakhalko (use `wandb login --relogin` to force relogin)


True

In [13]:
#!g1.1
wandb.init(entity='i_pakhalko', project='koila-example',reinit=True)

In [14]:
#!g1.1
print(torch.cuda.memory_allocated() // 1e6)
print(torch.cuda.memory_reserved() // 1e6)

0.0
0.0


In [15]:
#!g1.1
!nvidia-smi

Fri Mar 25 22:36:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:8C:00.0 Off |                    0 |
| N/A   29C    P0    21W / 300W |      4MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
#!g1.1
torch.cuda.empty_cache()

In [17]:
#!g1.1
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(224),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 2000

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [18]:
#!g1.1
trainset

Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=224, interpolation=bilinear, max_size=None, antialias=None)
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [19]:
#!g1.1
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

n_classes = 100

class MyResnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18()
        self.model.fc = nn.Linear(512, n_classes)
    
    def forward(self, x):
        return self.model(x)
    
class GiantFC(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # B, 224 * 224 * 3 = 150528
            nn.Linear(150528, fc_hidden),
            nn.ReLU(),
            nn.Linear(fc_hidden, 1024),
            nn.ReLU(),
            nn.Linear(1024, n_classes)
        )
    def forward(self, x):
        return self.model(x)

if use_resnet:
    model = MyResnet().to(device)
else:
    model = GiantFC().to(device)
wandb.watch(model)

[]

In [20]:
#!g1.1
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
opt = optim.AdamW(model.parameters(), lr=0.001)

In [21]:
#!g1.1
from math import ceil

opt_steps = 300
epochs = ceil(opt_steps / len(trainloader))
print(f"epochs: {epochs}")
log_step = ceil(len(trainloader) / 10)
print(f"log interval: {log_step}")

epochs: 12
log interval: 3


In [22]:
#!g1.1

for epoch in range(1, epochs+1):
    running_loss = 0.0
    for idx, data in enumerate(trainloader, 1):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        if use_koila:
            inputs, labels = lazy(inputs, labels, batch=0)
            
        model.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()

        # print statistics
        running_loss += loss.item()
        if idx % log_step == 0:
            print(inputs.shape)
            print(labels.shape)
            print(f'[{epoch}, {idx:5d}] loss: {running_loss / log_step:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: CUDA out of memory. Tried to allocate 5.61 GiB (GPU 0; 31.75 GiB total capacity; 29.32 GiB already allocated; 999.50 MiB free; 29.33 GiB reserved in total by PyTorch)

[0;31mKernelOutOfMemory[0m: Kernel ran out of memory and has been restarted. If the restart fails, restart the kernel from the Kernel menu.
If the error persists, try choosing a different configuration or optimizing your code.

In [ ]:
#!g1.1
torch.cuda.memory_reserved() // 1e6

0.0

In [ ]:
#!g1.1
del model
del opt
del criterion
torch.cuda.empty_cache()

In [ ]:
#!g1.1
torch.cuda.empty_cache()